In [9]:
import pandas as pd
import numpy
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import torch
from sklearn.metrics import classification_report

In [10]:
# smishtankのデータで評価してみる
df = pd.read_csv("data/smishtank.csv",encoding_errors="ignore")
df["Labels"] = 1
df_text = df["MainText"]
df_labels = df["Labels"]

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_text(texts):
    return tokenizer(list(texts),padding=True,truncation=True,max_length=128,return_tensors="pt")

encording = tokenize_text(df_text)

In [12]:
# 学習済みモデルをロード
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
model.load_state_dict(torch.load("model/bert_detection_model_epoch10.pth"))

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(next(model.parameters()))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_841/2860955440.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We rec

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       device='cuda:0', requires_grad=True)


In [13]:
# データ準備
# ラベルをテンソルに変換
labels = torch.tensor(df_labels.values)
# データローダー
dataset = torch.utils.data.TensorDataset(encording['input_ids'],encording['attention_mask'],labels)

loader = torch.utils.data.DataLoader(dataset,batch_size=16)


In [23]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

model.eval()
all_preds,all_labels = [],[]

with torch.no_grad():
    for batch in loader:
        input_ids,attention_mask,labels = [b.to(device) for b in batch]
        outputs = model(input_ids,attention_mask = attention_mask)
        preds = torch.argmax(outputs.logits,dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels,all_preds,target_names = ["ham","spam"]))


              precision    recall  f1-score   support

         ham       0.00      0.00      0.00         0
        spam       1.00      0.95      0.98      1062

    accuracy                           0.95      1062
   macro avg       0.50      0.48      0.49      1062
weighted avg       1.00      0.95      0.98      1062



/home/ubuntu/miniconda3/envs/bert/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ubuntu/miniconda3/envs/bert/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ubuntu/miniconda3/envs/bert/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# 詳細
print(numpy.array(all_preds))
print(numpy.array(all_labels))
falied_index_list = []
for i in range(len(all_labels)):
    if all_preds[i] != all_labels[i]:
        falied_index_list.append(i)



[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


In [24]:
#敵対的攻撃
import nltk
from nltk.corpus import wordnet as wn

nltk.download('wordnet')
nltk.download('omw-1.4')

model.eval()

# 同義語を取得する関数
def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_',' '))
    return list(synonyms)

# 単語置き換えによる敵対的攻撃
def adversarial_attack(text, model, tokenizer, device=device):
    words = text.split()  # 文を単語に分割
    original_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    original_pred = model(**original_inputs).logits.argmax().item()

    for i, word in enumerate(words):
        synonyms = get_synonyms(word)
        for synonym in synonyms:
            # 単語を置き換えた新しい文を生成
            new_words = words[:i] + [synonym] + words[i+1:]
            new_text = " ".join(new_words)

            # モデルで予測
            inputs = tokenizer(new_text, return_tensors="pt", padding=True, truncation=True).to(device)
            pred = model(**inputs).logits.argmax().item()

            # 予測が変わったら成功
            if pred != original_pred:
                return new_text, original_pred, pred

    # 置き換えが成功しなかった場合
    return text, original_pred, original_pred


[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
df_result_semantic = pd.DataFrame(columns=["original","adversarial","original_label","adversarial_label"])

for i in range(len(df_text)):
    text = df_text.iloc[i]
    original_text,adv_text, new_label, original_label = adversarial_attack_multi(text, model, tokenizer, device,similarity_threshold=0.6)
# DataFrameに行を追加するにはpd.concatを使用
    new_row = pd.DataFrame([{
        "original": original_text,
        "adversarial": adv_text,
        "original_label": original_label,
        "adversarial_label": new_label
    }])
    df_result_semantic = pd.concat([df_result_semantic, new_row], ignore_index=True)


In [ ]:
sample_text = "please click this link"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

adv_text, original_label, new_label = adversarial_attack(sample_text, model, tokenizer, device)

print(f"Original Text: {sample_text}")
print(f"Adversarial Text: {adv_text}")
print(f"Original Label: {original_label}, New Label: {new_label}")

Original Text: dear dhaval  click httpgoiminimapp and download indiamart app for free to stay updated about enquiries  buy leads anytime  anywhere
Adversarial Text: dear dhaval  click httpgoiminimapp and download indiamart app for free to stay updated about enquiries  buy leads anytime  anywhere
Original Label: 1, New Label: 1
